In [65]:
import os
from pathlib import Path

import boto3
from botocore.config import Config

In [61]:
DATA = Path("C:/Users/hp/Desktop/datasets/urban-feature-extraction/inria")

In [121]:
file_names = [x.name for x in (DATA / "masks").glob("*.jpg")]

In [84]:
s3 = boto3.resource('s3')
bucket = s3.Bucket('urban-feature-extraction')

In [117]:
class DatasetUploader():
    def __init__(self, bucket_name):
        self.init_boto3()
        self.s3resource = boto3.resource('s3')
        self.bucket = self.s3resource.Bucket(bucket_name)

        self.prefix = {
            "train_image": "labelled/patches/images",
            "train_mask": "labelled/patches/masks",

            "test_image": "labelled/scenes/images",
            "test_mask": "labelled/scenes/masks",
        }
    
    def init_boto3(self, aws_dir = None):
        if not aws_dir: 
            aws_dir = (Path.home() / ".aws")
        os.environ["AWS_CONFIG_FILE"] = (aws_dir / "config").as_posix()
        #os.environ["AWS_SHARED_CREDENTIALS_FILE"] = (aws_dir / "credentials").as_posix()
    
    def list_files_in_bucket(self):
        for file_obj in self.bucket.objects.all():
            print(file_obj.key)
    
    def upload_patch(self, patch_path:Path, prefix:str):
        assert patch_path.exists() and patch_path.is_file()

        patch_key = (Path(prefix) / patch_path.name).as_posix()
        self.bucket.Object(patch_key).upload_file(patch_path)
        print(f"Uploaded to {self.bucket.name}/{patch_key}")
    
    def upload_train_pair(self, file_name, image_dir, mask_dir):
        self.upload_patch(
            patch_path = (image_dir / file_name),
            prefix = self.prefix["train_image"]
        )
        self.upload_patch(
            patch_path = (mask_dir / file_name),
            prefix = self.prefix["train_mask"]
        )

    def upload_test_pair(self, file_name, image_dir, mask_dir):
        self.upload_patch(
            patch_path = (image_dir / file_name),
            prefix = self.prefix["test_image"]
        )
        self.upload_patch(
            patch_path = (mask_dir / file_name),
            prefix = self.prefix["test_mask"]
        )   

In [118]:
uploader = DatasetUploader('urban-feature-extraction')

In [122]:
for file_name in file_names[:50]:
    uploader.upload_train_pair(
        file_name=file_name,
        image_dir=(DATA/"images"),
        mask_dir=(DATA/"masks")
    )

Uploaded to urban-feature-extraction/labelled/patches/images/00004249-2209-481b-b3fa-387b64e80bfe.jpg
Uploaded to urban-feature-extraction/labelled/patches/masks/00004249-2209-481b-b3fa-387b64e80bfe.jpg
Uploaded to urban-feature-extraction/labelled/patches/images/0002968f-c3a9-41be-b4d5-f4bcb9e36f90.jpg
Uploaded to urban-feature-extraction/labelled/patches/masks/0002968f-c3a9-41be-b4d5-f4bcb9e36f90.jpg
Uploaded to urban-feature-extraction/labelled/patches/images/0004b3fd-87dd-4619-b2b9-67ec654f637a.jpg
Uploaded to urban-feature-extraction/labelled/patches/masks/0004b3fd-87dd-4619-b2b9-67ec654f637a.jpg
Uploaded to urban-feature-extraction/labelled/patches/images/00082786-4816-49a1-b1dc-9e073408f3e8.jpg
Uploaded to urban-feature-extraction/labelled/patches/masks/00082786-4816-49a1-b1dc-9e073408f3e8.jpg
Uploaded to urban-feature-extraction/labelled/patches/images/00083190-9ff5-473d-a9c7-cd7108d14bb0.jpg
Uploaded to urban-feature-extraction/labelled/patches/masks/00083190-9ff5-473d-a9c7-cd